In [26]:
import pandas as pd
from statistics import mean, median
import sys
import pprint

In [27]:
pp = pprint.PrettyPrinter(indent=4)

In [28]:
ssr_result_file = '../RQ2/ssr_final_results.csv'

In [29]:
batch_sizes = [1, 2, 4, 8, 16]
batching_alg = ['BATCH4', 'BATCHBISECT', 'BATCHSTOP4']

In [30]:
projects = {'sonarqube.csv', 'geoserver.csv', 'jruby.csv', 'vagrant.csv', 'cloud_controller_ng.csv', 'graylog2-server.csv', 'rails.csv', 'puppet.csv', 'fog.csv', 'rubinius.csv', 'sufia.csv', 'orbeon-forms.csv', 'opal.csv', 'gradle.csv', 'heroku.csv', 'cloudify.csv', 'metasploit-framework.csv', 'open-build-service.csv', 'loomio.csv', 'concerto.csv'}

In [31]:
def str_to_list(s):
    if s == '[]':
        return [0]
    l = s[1:-1].split(', ')
    l = [int(x) for x in l]
    return l

In [32]:
def get_results(results):
    l = []
    
    for r in results:
        if r == 'passed':
            l.append(1)
        else:
            l.append(0)
            
    return l

In [33]:
ssr_delay_lib = {}
for p in projects:
    ssr_delay_lib[p] = {'BATCHSTOP4':{4:{}, 8:{}, 16:{}}, 'BATCH4':{4:{}}, 'BATCHBISECT':{1:{}, 2:{}, 4:{}, 8:{}, 16:{}}}

In [34]:
ssr_results = pd.read_csv(ssr_result_file)
project_test_file = {}

In [35]:
for p in projects:
    
    test_file = pd.read_csv('../data/test_data/'+p.split('.')[0]+'_test.csv')
    y_test = get_results(test_file['tr_status'].tolist())
    project_test_file[p] = y_test

In [37]:
for x in range(len(ssr_results)):
    row = ssr_results.iloc[x]
    ci = str_to_list(row['ci'])
    b = row['batch_size'] - 1
    
    y_test = project_test_file[row['project']+'.csv']
    
    i = 0        
    delay_list = []
    missed = []

    while i < len(ci):

        sbs_delay = 0
        batch_delay = 0

        if ci[i] == 0:
            while len(missed) > 0:
                sbs_delay += i - missed.pop()
            batch_delay += b
            b -= 1
            if b == -1:
                b = row['batch_size'] - 1

        if ci[i] == 1:
            if y_test[i] == 0:
                missed.append(i)
                
        delay_list.append(sbs_delay + batch_delay)

        i += 1
    #if row['project'] == 'heroku' and row['batch_size']==4:
        #print('-----------------')
    point = {'builds_reqd':row['project_reqd_builds'], 'median_delay':median(delay_list), 'total_delay':sum(delay_list)}
    ssr_delay_lib[row['project']+'.csv'][row['algorithm']][row['batch_size']][row['confidence']] = point

In [38]:
ssr_delay_lib

{'geoserver.csv': {'BATCHSTOP4': {4: {2: {'builds_reqd': 95.83333333333331,
     'median_delay': 1.0,
     'total_delay': 242},
    3: {'builds_reqd': 80.95238095238095,
     'median_delay': 1.0,
     'total_delay': 224},
    4: {'builds_reqd': 75.5952380952381,
     'median_delay': 1.0,
     'total_delay': 223},
    5: {'builds_reqd': 76.78571428571429,
     'median_delay': 1.0,
     'total_delay': 220},
    6: {'builds_reqd': 80.95238095238095,
     'median_delay': 1.0,
     'total_delay': 233},
    7: {'builds_reqd': 71.42857142857143,
     'median_delay': 1.0,
     'total_delay': 229},
    8: {'builds_reqd': 81.54761904761905,
     'median_delay': 1.0,
     'total_delay': 232},
    9: {'builds_reqd': 75.5952380952381,
     'median_delay': 1.0,
     'total_delay': 232},
    10: {'builds_reqd': 74.4047619047619,
     'median_delay': 1.0,
     'total_delay': 239},
    11: {'builds_reqd': 67.85714285714286,
     'median_delay': 1.0,
     'total_delay': 233},
    12: {'builds_reqd': 67.

In [39]:
lines = []
for p in ssr_delay_lib:
    for alg in ssr_delay_lib[p]:
        for b in ssr_delay_lib[p][alg]:
            for c in ssr_delay_lib[p][alg][b]:
                d = ssr_delay_lib[p][alg][b][c]
                lines.append([p, 'ssr', alg, b, c, d['builds_reqd'], d['median_delay'], d['total_delay']])

In [40]:
df = pd.DataFrame(lines, columns=['project', 'method', 'algorithm', 'batch_size', 'confidence', 'builds_reqd', 'median_delays', 'total_delays'])


In [41]:
df.to_csv('ssr_confidence_all_projects_median.csv')